In [27]:
import sympy as sy
import numpy as np
import casadi as ca
import pathlib
from SE3Lie import *
import numpy as np
from sympy import *
from pyecca.lie import so3, r3, se3
from pyecca.lie.util import DirectProduct
from pyecca.lie.so3 import Quat, Dcm, Euler, Mrp
from pyecca.lie.r3 import R3
from pyecca.lie.se3 import SE3
from pyecca.test import test_lie
import casadi as ca
from SE3Lie import se3, SE3, se3_diff_correction_inv, dot_plot_draw
from casadi.tools.graph import graph, dotdraw, dotsave
import os
import pydot
from Sympy_To_Casadi import SympyToCasadi


Sympy for U_inv Matrix for Reference

In [28]:
x,y,z,theta_1,theta_2,theta_3 = symbols('x,y,z,theta_1,theta_2,theta_3',real=True)
theta = symbols('theta', real = True)
ad = sy.Matrix( [[0, -theta_3, theta_2, 0, -z, y], [theta_3, 0, -theta_1, z,0, -x], [-theta_2, theta_1,0,-y, x, 0], [0,0,0,0,-theta_3,theta_2],[0,0,0,theta_3,0,-theta_1],[0,0,0,-theta_2,theta_1,0]])
I = eye(6)
A, B = symbols('A, B', real=True)
UInv = (I + A*ad + B*ad**2) #separate into Odd and Even terms
ad

Matrix([
[       0, -theta_3,  theta_2,        0,       -z,        y],
[ theta_3,        0, -theta_1,        z,        0,       -x],
[-theta_2,  theta_1,        0,       -y,        x,        0],
[       0,        0,        0,        0, -theta_3,  theta_2],
[       0,        0,        0,  theta_3,        0, -theta_1],
[       0,        0,        0, -theta_2,  theta_1,        0]])

In [35]:
k = symbols('k',real=True)
ASum = summation( ((-1) ** k) *theta**(2*k) / (factorial(2*k+2)),(k,0,oo))
Asub = simplify(ASum) #A term
BSum = summation( ((-1) ** k) *theta**(2*k) / (factorial(2*k+3)),(k,0,oo))
Bsub = simplify(BSum) #B term
UInv = UInv.applyfunc(lambda x: x.subs({A:Asub,B:Bsub}).simplify())
subTerms = {theta_2**2 + theta_3**2: theta**2 - theta_1**2, theta_1**2 + theta_3**2: theta**2 - theta_2**2, theta_1**2 + theta_2**2: theta**2 - theta_3**2}
UInv = UInv.applyfunc(lambda x: x.subs(subTerms).simplify())
UInv #U inverse with A and B terms substituted closed form



Matrix([
[              (theta**3 + (theta - sin(theta))*(-theta**2 + theta_1**2))/theta**3,  (theta*theta_3*(cos(theta) - 1) + theta_1*theta_2*(theta - sin(theta)))/theta**3, (-theta*theta_2*(cos(theta) - 1) + theta_1*theta_3*(theta - sin(theta)))/theta**3,                            -2*(theta - sin(theta))*(theta_2*y + theta_3*z)/theta**3,  (theta*z*(cos(theta) - 1) + (theta - sin(theta))*(theta_1*y + theta_2*x))/theta**3, (-theta*y*(cos(theta) - 1) + (theta - sin(theta))*(theta_1*z + theta_3*x))/theta**3],
[(-theta*theta_3*(cos(theta) - 1) + theta_1*theta_2*(theta - sin(theta)))/theta**3,               (theta**3 + (theta - sin(theta))*(-theta**2 + theta_2**2))/theta**3,  (theta*theta_1*(cos(theta) - 1) + theta_2*theta_3*(theta - sin(theta)))/theta**3, (-theta*z*(cos(theta) - 1) + (theta - sin(theta))*(theta_1*y + theta_2*x))/theta**3,                            -2*(theta - sin(theta))*(theta_1*x + theta_3*z)/theta**3,  (theta*x*(cos(theta) - 1) + (theta - sin(theta))*(theta_2*z + th

U Inv matrix in casadi form and dot draw plot


parser = SympyToCasadi()
UInv_ca = parser.parse(UInv, cse=True)
UInv_ca

In [36]:
#Without subterms for theta
parser = SympyToCasadi()
UInv_ca = parser.parse(UInv, cse=True)
UInv_ca

SX(@1=(theta*sq(theta)), @2=(theta-sin(theta)), @3=sq(theta), @4=((@1+(@2*(sq(theta_1)-@3)))/@1), @5=(theta*(-1+cos(theta))), @6=(@5*theta_3), @7=((((@2*theta_1)*theta_2)-@6)/@1), @8=(@5*theta_2), @9=(@2*theta_1), @10=((@8+(@9*theta_3))/@1), @11=0, @12=((@6+(@9*theta_2))/@1), @13=((@1+(@2*(sq(theta_2)-@3)))/@1), @14=(@5*theta_1), @15=((((@2*theta_2)*theta_3)-@14)/@1), @16=((((@2*theta_1)*theta_3)-@8)/@1), @17=((@14+((@2*theta_2)*theta_3))/@1), @18=((@1+(@2*(sq(theta_3)-@3)))/@1), @19=((2/@1)*@2), @20=(theta_2*y), @21=(theta_3*z), @22=((theta_1*y)+(theta_2*x)), @23=(@5*z), @24=(@5*y), @25=((theta_1*z)+(theta_3*x)), @26=(theta_1*x), @27=((theta_2*z)+(theta_3*y)), @28=(@5*x), 
[[@4, @12, @16, (-(@19*(@20+@21))), ((@23+(@22*@2))/@1), (((@25*@2)-@24)/@1)], 
 [@7, @13, @17, (((@22*@2)-@23)/@1), (-(@19*(@21+@26))), ((@28+(@2*@27))/@1)], 
 [@10, @15, @18, ((@24+(@25*@2))/@1), (((@2*@27)-@28)/@1), (-(@19*(@20+@26)))], 
 [@11, @11, @11, @4, @12, @16], 
 [@11, @11, @11, @7, @13, @17], 
 [@11, @11

In [39]:
dot_plot_draw(UInv_ca)